In [1]:
import pyspark
from pyspark.sql import SparkSession
import pandas as pd

In [2]:
#create SparkSession
spark = SparkSession.builder\
.master("local[1]")\
.appName("Github")\
.config('spark.ui.port', '4052')\
.getOrCreate()
spark

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/12/29 18:45:56 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/12/29 18:45:59 WARN Utils: Service 'SparkUI' could not bind on port 4052. Attempting port 4053.


In [3]:
#import Dataframe
dataset=spark.read.parquet('/Users/nicolaroth/Downloads/ne3_small_subset.parquet')

In [4]:
#set all columns as doubletype
for col in dataset.columns:
    dataset = dataset.withColumn(col,dataset[col].cast('double'))

In [5]:
#imput mean in NULL-Values
#import
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import Imputer
from pyspark.ml.feature import MinMaxScaler
from pyspark.ml.classification import MultilayerPerceptronClassifier

In [6]:
#reorder columns
dataset = dataset.select(sorted(dataset.columns, reverse=True))

In [7]:
#create model for imputation with mean
imputed_col = ['UW{}'.format(10+i) for i in range(0, 15)]

model = Imputer(strategy='mean',missingValue=None,inputCols=imputed_col,outputCols=imputed_col).fit(dataset)
impute_data = model.transform(dataset)

assemble = VectorAssembler(inputCols=imputed_col, outputCol='assembled_features', handleInvalid='error')
a_data = assemble.transform(impute_data)
scaler = MinMaxScaler(min=0.0, max=1.0, inputCol='assembled_features', outputCol='features')
s_data = scaler.fit(a_data).transform(a_data)

a = ['UW10', 'UW11', 'UW12', 'UW13', 'UW14', 'UW15', 'UW16', 'UW17', 'UW18', 'UW19', 'UW20', 'UW21', 'UW22', 'UW23', 'UW24','features']
s_data.select(*a) . show()

22/12/29 18:46:15 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+------------------+------------------+------------------+------------------+------------------+------------------+-----------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+--------------------+
|              UW10|              UW11|              UW12|              UW13|              UW14|              UW15|             UW16|              UW17|              UW18|              UW19|              UW20|              UW21|              UW22|              UW23|              UW24|            features|
+------------------+------------------+------------------+------------------+------------------+------------------+-----------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+--------------------+
|          17080.32|            5337.6| 8139.840000000001|           8273.28|12

In [8]:
#execute model
train_df,test_df = s_data.select('UW10', 'UW11', 'UW12', 'UW13', 'UW14', 'UW15', 'UW16', 'UW17', 'UW18', 'UW19', 'UW20', 'UW21', 'UW22', 'UW23', 'UW24','features').randomSplit([0.7,0.3],1213)

mlpc=MultilayerPerceptronClassifier(featuresCol='features', labelCol = 'UW11', layers = [16,16,2],\
 maxIter=1000,blockSize=8,seed=7,solver='gd')
ann = mlpc.fit(train_df)

22/12/29 18:46:28 WARN BlockManager: Putting block rdd_40_0 failed due to exception org.apache.spark.SparkException: Failed to execute user defined function (OneHotEncoderModel$$Lambda$3641/2132759836: (double, int) => struct<type:tinyint,size:int,indices:array<int>,values:array<double>>).
22/12/29 18:46:28 WARN BlockManager: Block rdd_40_0 could not be removed as it was not found on disk or in memory
22/12/29 18:46:28 ERROR Executor: Exception in task 0.0 in stage 9.0 (TID 7)
org.apache.spark.SparkException: Failed to execute user defined function (OneHotEncoderModel$$Lambda$3641/2132759836: (double, int) => struct<type:tinyint,size:int,indices:array<int>,values:array<double>>)
	at org.apache.spark.sql.errors.QueryExecutionErrors$.failedExecuteUserDefinedFunctionError(QueryExecutionErrors.scala:190)
	at org.apache.spark.sql.errors.QueryExecutionErrors.failedExecuteUserDefinedFunctionError(QueryExecutionErrors.scala)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$Generate

Py4JJavaError: An error occurred while calling o277.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 9.0 failed 1 times, most recent failure: Lost task 0.0 in stage 9.0 (TID 7) (nicolas-macbook-air.home executor driver): org.apache.spark.SparkException: Failed to execute user defined function (OneHotEncoderModel$$Lambda$3641/2132759836: (double, int) => struct<type:tinyint,size:int,indices:array<int>,values:array<double>>)
	at org.apache.spark.sql.errors.QueryExecutionErrors$.failedExecuteUserDefinedFunctionError(QueryExecutionErrors.scala:190)
	at org.apache.spark.sql.errors.QueryExecutionErrors.failedExecuteUserDefinedFunctionError(QueryExecutionErrors.scala)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anon$1.hasNext(W